In [34]:
import requests
from bs4 import BeautifulSoup
import time 
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import math
import pandas as pd
import warnings
import pickle
import GetSalary
import importlib
import scrape2_2 as scr2
import driver_config as dc
import numpy as np
import scipy.interpolate as si
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
importlib.reload(scr2)
importlib.reload(dc)


<module 'driver_config' from 'c:\\Users\\ckelaid\\Documents\\Scraping\\driver_config.py'>

In [35]:
# Get hrefs
# BI Analyst: https://www.glassdoor.com/Job/bi-analyst-jobs-SRCH_KO0,10.htm?clickSource=searchBox
# Data Engineer: https://www.glassdoor.com/Job/bi-analyst-jobs-SRCH_KO0,13.htm
hrefs = []
p = 0
browser = dc.initialize_driver()
browser.get('https://www.glassdoor.com/Job/bi-analyst-jobs-SRCH_KO0,10.htm?clickSource=searchBox')
while p <= 15:
    try:
        a_tags = browser.find_element(By.ID, 'MainCol').find_elements(By.TAG_NAME, 'a')#[0].get_attribute('href')
        for a in a_tags:
            hrefs.append(a.get_attribute('href'))
        
        print("p is: ", p)
        # Click for next page
        page_button = browser.find_element(By.XPATH, "//button[@class='page  css-1hq9k8 e13qs2071']")
        page_button.click()
        page_button.send_keys(Keys.TAB) # move to the next page button
        p += 1 # Once p bigger than 3 (so 4 pages) this stops 
    
    except Exception:
        browser.implicitly_wait(10)

browser.quit()



p is:  0
p is:  1
p is:  2
p is:  3
p is:  4
p is:  5
p is:  6
p is:  7
p is:  8
p is:  9
p is:  10
p is:  11
p is:  12
p is:  13
p is:  14
p is:  15


In [36]:
len(hrefs)

2966

In [37]:
hrefs = list(set(hrefs)) # get unique hrefs only
new_hrefs = []
for h in hrefs:
    if h.startswith('https://www.glassdoor.com/partner/jobListing.htm?'):
        new_hrefs.append(h)

len(new_hrefs) # 120 => 30 per page

437

In [38]:
len(new_hrefs)

437

In [39]:
# Grab job info from job_links

def get_job_info(keyword:str, hrefs:list) -> list:

    jobs = []
    # t = random.randint(8,10)
    # WebDriverWait(browser, t)
    for h in new_hrefs:
        browser = dc.initialize_driver()
        t = random.randint(8,10)
        browser.implicitly_wait(t)
        browser.get(h)
        try:
            # Wait for elements to load
            WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='css-1rzz8ht ecgq1xb2']")))
            # Click show more first
            browser.find_element(By.XPATH, "//div[@class='css-1rzz8ht ecgq1xb2']").click()
            # job info
            job_info = browser.find_element(By.ID, 'JobView').text.split('\n')
            jobs.append(job_info)
            browser.quit()
            with open(keyword+'.pkl', 'wb') as f:
                pickle.dump(jobs, f)
        except Exception:
            try:
                job_info = browser.find_element(By.ID, 'JobView').text.split('\n')
                jobs.append(job_info)
                browser.quit()
                with open(keyword+'.pkl', 'wb') as f:
                    pickle.dump(jobs, f)
            except Exception:
                print("Can't retrieve this job's info")
                browser.quit()
    
    return jobs

In [40]:
def save_csv(keyword:str, job_dict:dict) :

    df = pd.DataFrame(job_dict.values(), columns = ['Job Info'], index = job_dict.keys())

    df.to_csv(keyword + '_Jobs.csv', index=False)

In [41]:
def scrape(keyword:str, hrefs:list):

    job_infos = get_job_info(keyword, hrefs)

    job_dict = {}
    for i in range(len(job_infos)):
        job_dict['Job ' + str(i+1)] = ' '.join(job_infos[i])
    
    save_csv(keyword, job_dict)

In [42]:
scrape('BI_Analyst', new_hrefs)

-----------------------------------------------------------------------------------------

In [23]:
len(jobs)

234

In [25]:
#jobs = []
    # t = random.randint(8,10)
    # WebDriverWait(browser, t)
for h in new_hrefs[234:]:
    browser = dc.initialize_driver()
    t = random.randint(8,10)
    browser.implicitly_wait(t)
    browser.get(h)
    try:
        # Wait for elements to load
        WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='css-1rzz8ht ecgq1xb2']")))
        # Click show more first
        browser.find_element(By.XPATH, "//div[@class='css-1rzz8ht ecgq1xb2']").click()
        # job info
        job_info = browser.find_element(By.ID, 'JobView').text.split('\n')
        jobs.append(job_info)
        browser.quit()
        with open('Data_Engineer.pkl', 'wb') as f:
            pickle.dump(jobs, f)
    except Exception:
        try:
            job_info = browser.find_element(By.ID, 'JobView').text.split('\n')
            jobs.append(job_info)
            browser.quit()
            with open('BI_Analyst.pkl', 'wb') as f:
                pickle.dump(jobs, f)
        except Exception:
            print("Can't retrieve this job's info")
            browser.quit()

In [26]:
len(jobs)

480

In [27]:
job_dict = {}
for i in range(len(jobs)):
    job_dict['Job ' + str(i+1)] = ' '.join(jobs[i])

In [29]:
save_csv('Data_Engineer', job_dict)

In [117]:
job_infos = get_job_info(new_hrefs)

In [124]:
job_dict = {}
for i in range(len(job_infos)):
    job_dict['Job ' + str(i+1)] = ' '.join(job_infos[i])

-------------------------------------------------------------------------